<a href="https://colab.research.google.com/github/AyHaski/DL_AtariRainbow/blob/master/Anyrl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install anyrl 

     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 8.9MB 7.1MB/s 
     |████████████████████████████████| 1.5MB 40.4MB/s 
  Created wheel for gym: filename=gym-0.10.11-cp36-none-any.whl size=1588314 sha256=e74c1159a24a6b01b9a1bf62c4d2924560b963fef773148d92c0bc59c97b0ba0
  Stored in directory: /root/.cache/pip/wheels/7b/eb/1f/22c4124f3c64943aa0646daf4612b1c1f00f27d89b81304ebd
Successfully built gym
ERROR: google-colab 1.0.0 has requirement pandas~=0.25.0; python_version >= "3.0", but you'll have pandas 0.23.4 which is incompatible.
  Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3
  Found existing installation: gym 0.15.4
    Uninstalling gym-0.15.4:
      Successfully uninstalled gym-0.15.4


In [0]:
RUN='3'
RUN_NUM='3_1'

"""
Hyperparameters

"""
lr=6.25e-5
lr_epsilon=1.5e-4
num_atoms=51
v_min=-10
v_max=10
noisy_sigma=0.5
n_step=3
#dueling=True
min_buffer_size=20000
num_steps=2000000
buffer_size=500000
#workers=8
target_interval=8192
batch_size=32
replay_epsilon=0.0
replay_alpha=0.5
replay_beta=0.4
game='SpaceInvaders'
restore=False

In [0]:
#@title Modified Replay Buffer Code
"""
Various replay buffer implementations.
"""

from math import sqrt
import random
import pickle
from anyrl.rollouts import ReplayBuffer
import numpy as np

class ModifiedPrioritizedReplayBuffer(ReplayBuffer):
    """
    A prioritized replay buffer with loss-proportional
    sampling.
    Weights passed to add_sample() and update_weights()
    are assumed to be error terms (e.g. the absolute TD
    error).
    """

    def __init__(self, capacity, alpha, beta, first_max=1, epsilon=0):
        """
        Create a prioritized replay buffer.
        The beta parameter can be any object that has
        support for the float() built-in.
        This way, you can use a TFScheduleValue.
        Args:
          capacity: the maximum number of transitions to
            store in the buffer.
          alpha: an exponent controlling the temperature.
            Higher values result in more prioritization.
            A value of 0 yields uniform prioritization.
          beta: an exponent controlling the amount of
            importance sampling. A value of 1 yields
            unbiased sampling. A value of 0 yields no
            importance sampling.
          first_max: the initial weight for new samples
            when no init_weight is specified and the
            buffer is completely empty.
          epsilon: a value which is added to every error
            term before the error term is used.
        """
        self.capacity = capacity
        self.alpha = alpha
        self.beta = beta
        self.epsilon = epsilon
        self.transitions = []
        self.errors = FloatBuffer(capacity)
        self._max_weight_arg = first_max

    @property
    def size(self):
        return len(self.transitions)

    def sample(self, num_samples):
        indices, probs = self.errors.sample(num_samples)
        beta = float(self.beta)
        importance_weights = np.power(probs * self.size, -beta)
        importance_weights /= np.power(self.errors.min() / self.errors.sum() * self.size, -beta)
        samples = []
        for i, weight in zip(indices, importance_weights):
            sample = self.transitions[i].copy()
            sample['weight'] = weight
            sample['id'] = i
            samples.append(sample)
        return samples

    def add_sample(self, sample, init_weight=None):
        """
        Add a sample to the buffer.
        When new samples are added without an explicit
        initial weight, the maximum weight argument ever
        seen is used. When the buffer is empty, first_max
        is used.
        """
        self.transitions.append(sample)
        if init_weight is None:
            self.errors.append(self._process_weight(self._max_weight_arg))
        else:
            self.errors.append(self._process_weight(init_weight))
        while len(self.transitions) > self.capacity:
            del self.transitions[0]

    def update_weights(self, samples, new_weights):
        for sample, weight in zip(samples, new_weights):
            self.errors.set_value(sample['id'], self._process_weight(weight))

    def _process_weight(self, weight):
        self._max_weight_arg = max(self._max_weight_arg, weight)
        return (weight + self.epsilon) ** self.alpha

    #saving replay buffer to pickle file
    def save_samples(self):
        print("saving transitions: ",len(self.transitions))
        with open('/content/drive/My Drive/AN/transitions'+RUN_NUM+'.p','wb') as handle:
            pickle.dump(self.transitions,handle,protocol=pickle.HIGHEST_PROTOCOL)

    #loading replay buffer from pickle file
        #have to add sample by sample due to populating the errors into the FloatBuffer (self.errors)
    def load_samples(self):
        temp_trans = pickle.load(open("/content/drive/My Drive/AN/transitions"+RUN_NUM+".p","rb"))
        for i in temp_trans:
            self.add_sample(i)
        print("loaded transitions: ",len(self.transitions))

class FloatBuffer:
    """A ring-buffer of floating point values."""

    def __init__(self, capacity, dtype='float64'):
        self._capacity = capacity
        self._start = 0
        self._used = 0
        self._buffer = np.zeros((capacity,), dtype=dtype)
        self._bin_size = int(sqrt(capacity))
        num_bins = capacity // self._bin_size
        if num_bins * self._bin_size < capacity:
            num_bins += 1
        self._bin_sums = np.zeros((num_bins,), dtype=dtype)
        self._min = 0

    def append(self, value):
        """
        Add a value to the end of the buffer.
        If the buffer is full, the first value is removed.
        """
        idx = (self._start + self._used) % self._capacity
        if self._used < self._capacity:
            self._used += 1
        else:
            self._start = (self._start + 1) % self._capacity
        self._set_idx(idx, value)

    def sample(self, num_values):
        """
        Sample indices in proportion to their value.
        Returns:
          A tuple (indices, probs)
        """
        assert self._used >= num_values
        res = []
        probs = []
        bin_probs = self._bin_sums / np.sum(self._bin_sums)
        while len(res) < num_values:
            bin_idx = np.random.choice(len(self._bin_sums), p=bin_probs)
            bin_values = self._bin(bin_idx)
            sub_probs = bin_values / np.sum(bin_values)
            sub_idx = np.random.choice(len(bin_values), p=sub_probs)
            idx = bin_idx * self._bin_size + sub_idx
            res.append(idx)
            probs.append(bin_probs[bin_idx] * sub_probs[sub_idx])
        return (np.array(list(res)) - self._start) % self._capacity, np.array(probs)

    def set_value(self, idx, value):
        """Set the value at the given index."""
        idx = (idx + self._start) % self._capacity
        self._set_idx(idx, value)

    def min(self):
        """Get the minimum value in the buffer."""
        return self._min

    def sum(self):
        """Get the sum of the values in the buffer."""
        return np.sum(self._bin_sums)

    def _set_idx(self, idx, value):
        assert not np.isnan(value)
        assert value > 0
        needs_recompute = False
        if self._min == self._buffer[idx]:
            needs_recompute = True
        elif value < self._min:
            self._min = value
        bin_idx = idx // self._bin_size
        self._buffer[idx] = value
        self._bin_sums[bin_idx] = np.sum(self._bin(bin_idx))
        if needs_recompute:
            self._recompute_min()

    def _bin(self, bin_idx):
        if bin_idx == len(self._bin_sums) - 1:
            return self._buffer[self._bin_size * bin_idx:]
        return self._buffer[self._bin_size * bin_idx:self._bin_size * (bin_idx + 1)]

    def _recompute_min(self):
        if self._used < self._capacity:
            self._min = np.min(self._buffer[:self._used])
        else:
            self._min = np.min(self._buffer)

In [0]:
#@title Modified DQN Code
%tensorflow_version 1.x
import time

import tensorflow as tf


class DQN:
    """
    Train TFQNetwork models using Q-learning.
    """

    def __init__(self, online_net, target_net, discount=0.99):
        """
        Create a Q-learning session.
        Args:
          online_net: the online TFQNetwork.
          target_net: the target TFQNetwork.
          discount: the per-step discount factor.
        """
        self.online_net = online_net
        self.target_net = target_net
        self.discount = discount

        obs_shape = (None,) + online_net.obs_vectorizer.out_shape
        self.obses_ph = tf.placeholder(online_net.input_dtype, shape=obs_shape)
        self.actions_ph = tf.placeholder(tf.int32, shape=(None,))
        self.rews_ph = tf.placeholder(tf.float32, shape=(None,))
        self.new_obses_ph = tf.placeholder(online_net.input_dtype, shape=obs_shape)
        self.terminals_ph = tf.placeholder(tf.bool, shape=(None,))
        self.discounts_ph = tf.placeholder(tf.float32, shape=(None,))
        self.weights_ph = tf.placeholder(tf.float32, shape=(None,))

        losses = online_net.transition_loss(target_net, self.obses_ph, self.actions_ph,
                                            self.rews_ph, self.new_obses_ph, self.terminals_ph,
                                            self.discounts_ph)
        self.losses = self.weights_ph * losses
        self.loss = tf.reduce_mean(self.losses)

        assigns = []
        for dst, src in zip(target_net.variables, online_net.variables):
            assigns.append(tf.assign(dst, src))
        self.update_target = tf.group(*assigns)

    def feed_dict(self, transitions):
        """
        Generate a feed_dict that feeds the batch of
        transitions to the DQN loss terms.
        Args:
          transition: a sequence of transition dicts, as
            defined in anyrl.rollouts.ReplayBuffer.
        Returns:
          A dict which can be fed to tf.Session.run().
        """
        obs_vect = self.online_net.obs_vectorizer
        res = {
            self.obses_ph: obs_vect.to_vecs([t['obs'] for t in transitions]),
            self.actions_ph: [t['model_outs']['actions'][0] for t in transitions],
            self.rews_ph: [self._discounted_rewards(t['rewards']) for t in transitions],
            self.terminals_ph: [t['new_obs'] is None for t in transitions],
            self.discounts_ph: [(self.discount ** len(t['rewards'])) for t in transitions],
            self.weights_ph: [t['weight'] for t in transitions]
        }
        new_obses = []
        for trans in transitions:
            if trans['new_obs'] is None:
                new_obses.append(trans['obs'])
            else:
                new_obses.append(trans['new_obs'])
        res[self.new_obses_ph] = obs_vect.to_vecs(new_obses)
        return res

    def optimize(self, learning_rate=6.25e-5, epsilon=1.5e-4, **adam_kwargs):
        """
        Create a TF Op that optimizes the objective.
        Args:
          learning_rate: the Adam learning rate.
          epsilon: the Adam epsilon.
        """
        optim = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=epsilon, **adam_kwargs)
        return optim.minimize(self.loss)

    def train(self,
              num_steps,
              player,
              replay_buffer,
              optimize_op,
              train_interval=1,
              target_interval=8192,
              batch_size=32,
              min_buffer_size=20000,
              tf_schedules=(),
              handle_ep=lambda steps, rew: None,
              timeout=None):
        """
        Run an automated training loop.
        This is meant to provide a convenient way to run a
        standard training loop without any modifications.
        You may get more flexibility by writing your own
        training loop.
        Args:
          num_steps: the number of timesteps to run.
          player: the Player for gathering experience.
          replay_buffer: the ReplayBuffer for experience.
          optimize_op: a TF Op to optimize the model.
          train_interval: timesteps per training step.
          target_interval: number of timesteps between
            target network updates.
          batch_size: the size of experience mini-batches.
          min_buffer_size: minimum replay buffer size
            before training is performed.
          tf_schedules: a sequence of TFSchedules that are
            updated with the number of steps taken.
          handle_ep: called with information about every
            completed episode.
          timeout: if set, this is a number of seconds
            after which the training loop should exit.
        """
        
        
        sess = self.online_net.session
        
        tnrewsph = tf.summary.scalar(name='rews_ph', tensor=tf.reduce_mean(self.rews_ph))
        tndiscountsph = tf.summary.scalar(name='discounts_ph', tensor=tf.reduce_mean(self.discounts_ph))
        tnweightsph = tf.summary.scalar(name='weights_ph', tensor=tf.reduce_mean(self.weights_ph))
        tnlosses = tf.summary.scalar(name='losses', tensor=tf.reduce_mean(self.losses))

        merge = tf.summary.merge([tnrewsph,tndiscountsph,tnweightsph,tnlosses])
        train_writer = tf.summary.FileWriter( '/content/drive/My Drive/AN/logs/'+ RUN_NUM +'/train', sess.graph)
        
        sess.run(self.update_target)
        steps_taken = 0
        next_target_update = target_interval
        next_train_step = train_interval
        start_time = time.time()

        if restore:
          replay_buffer.load_samples()

        while steps_taken < num_steps:
            if timeout is not None and time.time() - start_time > timeout:
                return
            transitions = player.play()
            for trans in transitions:
                if trans['is_last']:
                    handle_ep(trans['episode_step'] + 1, trans['total_reward'], trans['episode_id'])
                replay_buffer.add_sample(trans)
                steps_taken += 1
                for sched in tf_schedules:
                    sched.add_time(sess, 1)
                if replay_buffer.size >= min_buffer_size and steps_taken >= next_train_step:
                    next_train_step = steps_taken + train_interval
                    batch = replay_buffer.sample(batch_size)
                    
                    #_, losses = sess.run((optimize_op, self.losses),
                    #                     feed_dict=self.feed_dict(batch))
                    _, losses, summary = sess.run((optimize_op, self.losses, merge),
                                         feed_dict=self.feed_dict(batch))
                    
                    train_writer.add_summary(summary, steps_taken)
                    replay_buffer.update_weights(batch, losses)
                    
                if steps_taken >= next_target_update:
                    next_target_update = steps_taken + target_interval
                    sess.run(self.update_target)
                if (steps_taken % 100000 ==0):
                   replay_buffer.save_samples()

    def _discounted_rewards(self, rews):
        res = 0
        for i, rew in enumerate(rews):
            res += rew * (self.discount ** i)
        return res

In [0]:
#@title para agrs don't work 
import argparse

def _parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr', help='Adam learning rate', type=float, default=6.25e-5)
    parser.add_argument('--lr-epsilon', help='Adam epsilon', type=float, default=1.5e-4)
    parser.add_argument('--num-atoms', help='Number of distribution atoms', type=int, default=51)
    parser.add_argument('--v-min', help='Minimum atom value', type=int, default=-10)
    parser.add_argument('--v-max', help='Maximum atom value', type=int, default=10)
    parser.add_argument('--noisy-sigma', help='initial Noisy Net noise', type=float, default=0.5)
    parser.add_argument('--n-step', help='Number of multi-step', type=int, default=3)
    #parser.add_argument('--dueling', help='Set dueling architecture', type=bool, default=True)
    parser.add_argument('--min-buffer-size', help='replay buffer size before training',
                        type=int, default=20000)
    parser.add_argument('--num-steps', help='Number of max steps to train', type=int, default=2000000)
    parser.add_argument('--buffer-size', help='replay buffer size', type=int, default=1000000)
   #parser.add_argument('--workers', help='number of parallel envs', type=int, default=8)
    parser.add_argument('--target-interval', help='training iters per log', type=int, default=8192)
    parser.add_argument('--batch-size', help='SGD batch size', type=int, default=32)
    parser.add_argument('--replay-epsilon', help='initial epsilon', type=float, default=0.0)
    parser.add_argument('--replay-alpha', help='Replay alpha', type=float, default=0.5)
    parser.add_argument('--replay-beta', help='Replay beta', type=float, default=0.4)
    parser.add_argument('game', help='game name', default='SpaceInvaders')
    parser.add_argument('--restore', help='load from checkpoint', action='store_true')
    return parser.parse_args()

In [0]:
  tf.reset_default_graph()

In [8]:
#!/usr/bin/env python
%tensorflow_version 1.x

import tensorflow as tf
import os
import time
from gym.wrappers import Monitor
from functools import partial
from anyrl.envs import batched_gym_env
from anyrl.envs.wrappers import BatchedFrameStack, DownsampleEnv, GrayscaleEnv
from anyrl.models import rainbow_models
from anyrl.rollouts import BatchedPlayer, NStepPlayer
from anyrl.spaces import gym_space_vectorizer
from anyrl.utils import tf_state
import gym
import numpy as np

REWARD_HISTORY = 10

def wrap_env(env):
  env = Monitor(env, '/content/drive/My Drive/videoAnyrl/'+RUN_NUM, force=True )
  return env

def make_env():
    """
    Create an environment with some standard wrappers.
    """
    #env = retro.make(game='SpaceInvaders-Atari2600', record=True)
    env = wrap_env(gym.make(game+'-v0'))
    return GrayscaleEnv(DownsampleEnv(env, 2))

def main():
    
    env = batched_gym_env([partial(make_env)])

    env = BatchedFrameStack(env, num_images=4, concat=False)

    checkpoint_dir = os.path.join(os.getcwd(), 'drive/My Drive/ANresults')
    results_dir = os.path.join(os.getcwd(), 'drive/My Drive/ANresults', time.strftime("%d-%m-%Y_%H-%M-%S"))
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    summary_writer = tf.summary.FileWriter(results_dir)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True # pylint: disable=E1101
    
    
    with tf.Session(config=config) as sess:  
        
        dqn=DQN(*rainbow_models(sess,
                              env.action_space.n,
                              gym_space_vectorizer(env.observation_space),
                              num_atoms=num_atoms,
                              min_val=v_min,
                              max_val=v_max))

        saver = tf.train.Saver()
        #if restore:
         #   latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
         #   #saver = tf.train.import_meta_graph(checkpoint_dir)
         #   if latest_checkpoint:
         #       print("Loading model checkpoint {} ...\n".format(latest_checkpoint))
         #       saver.restore(sess, latest_checkpoint)
         #   else:
         #     print("Checkpoint not found")     


        player = NStepPlayer(BatchedPlayer(env, dqn.online_net), n_step)
        optimize = dqn.optimize(lr, lr_epsilon)
        sess.run(tf.global_variables_initializer())
        if(restore):
          tf_state.load_vars(sess,'/content/drive/My Drive/AN/anyrlM'+RUN_NUM)

        reward_hist = []
        total_steps = 0
        save_model_steps=100000
        episodes=0

        def _handle_ep(steps, rew, id):
            nonlocal total_steps
            nonlocal episodes
            nonlocal save_model_steps
            total_steps += steps
            episodes += 1
            reward_hist.append(rew)

            summary_reward = tf.Summary()
            summary_reward.value.add(tag='global/reward', simple_value=rew)
            summary_writer.add_summary(summary_reward, global_step=total_steps)

            if len(reward_hist) == REWARD_HISTORY:
              print('ID: %d | %d steps | %f mean' % (id,total_steps, sum(reward_hist) / len(reward_hist)))
            
              summary_meanreward = tf.Summary()
              summary_meanreward.value.add(tag='global/mean_reward', simple_value=sum(reward_hist) / len(reward_hist))
              summary_writer.add_summary(summary_meanreward, global_step=total_steps)

              reward_hist.clear()
            if(total_steps>= save_model_steps):
              save_model_steps +=100000
              print('save model')
              saver.save(sess=sess, save_path=checkpoint_dir + '/model', global_step=total_steps)
              tf_state.save_vars(sess,'/content/drive/My Drive/AN/anyrlM'+RUN_NUM)
                 
        
        dqn.train(num_steps=num_steps, 
                  player=player,
                  replay_buffer=ModifiedPrioritizedReplayBuffer(buffer_size, replay_alpha, replay_beta, epsilon=replay_epsilon),
                  optimize_op=optimize,
                  train_interval=1,
                  target_interval=target_interval,
                  batch_size=batch_size,
                  min_buffer_size=min_buffer_size,
                  timeout=3600*10,
                  handle_ep=_handle_ep)
        
    env.close()
    

if __name__ == '__main__':
      main()




Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ID: 9 | 7046 steps | 81.000000 mean
ID: 19 | 14111 steps | 95.000000 mean
ID: 29 | 21881 steps | 147.500000 mean
ID: 39 | 31031 steps | 192.000000 mean
ID: 49 | 39708 steps | 260.000000 mean
ID: 59 | 47573 steps | 244.000000 mean
ID: 69 | 55944 steps | 214.500000 mean
ID: 79 | 65295 steps | 263.500000 mean
ID: 89 | 72478 steps | 201.500000 mean
ID: 99 | 78947 steps | 171.500000 mean
ID: 109 | 85679 steps | 171.000000 mean
ID: 119 | 94768 steps | 337.000000 mean
saving transitions:  100000
save model
ID: 129 | 101020 steps | 164.500000 mean
ID: 139 | 108215 steps | 171.500000 mean
ID: 149 | 115721 steps | 219.500000 mean
ID: 159 | 123039 steps | 244.500000 mean
ID: 169 | 130561 steps | 239.000000 mean
ID: 179 | 139176 steps | 262.500000 mean
ID: 189 | 14

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in double_scalars


AssertionError: ignored

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-12-14 12:31:24--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.206.126.139, 3.229.196.117, 52.73.147.107, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.206.126.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.4MB/s    in 0.9s    

2019-12-14 12:31:25 (14.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = '/content/drive/My Drive/ANresults'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://fc05fe3e.ngrok.io


In [0]:
!tensorboard --logdir='/content/drive/My Drive/ANresults' 

TensorBoard 1.15.0 at http://482ce1a6b003:6006/ (Press CTRL+C to quit)
